In [ ]:
from glob import glob
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image, ImageDraw
import json
import numpy as np

# Show Ground Truth (plot original, ground truth, emphasizing the area)

In [ ]:
test_file_path = './data/original_final/test'

In [ ]:
want_to_check = list(map(os.path.basename, glob(os.path.join(test_file_path, '*.png'))))

In [ ]:
check_file = want_to_check[6]

plt.figure(figsize=(20,20))

# Plot DRR Image without Nodules
a0 = plt.subplot(1, 3, 1)
plt.imshow(Image.open(os.path.join(original_file_path, check_file)), cmap='gray')
a0.axis(False)


# Plot DRR Image with Nodules
a1 = plt.subplot(1, 3, 2)
img = Image.open(os.path.join(original_file_path, check_file))

with open('./test_revised_opencxr.json') as json_read:
    json_data = json.load(json_read)

img_idx = 0
for idx in range(len(json_data['images'])):
    if json_data['images'][idx]['file_name'] == check_file:
        img_idx = json_data['images'][idx]['id']
        break

anno_idx = []
for idx in range(len(json_data['annotations'])):
    if json_data['annotations'][idx]['image_id'] == img_idx:
        anno_idx.append(idx)

for idx in anno_idx:
    row = json_data['annotations'][idx]
    bbox_x, bbox_y, bbox_w, bbox_h = row['bbox']
    print(bbox_x, bbox_y, bbox_w, bbox_h)
    rect = patches.Rectangle((bbox_x, bbox_y), bbox_w, bbox_h, linewidth=2, edgecolor='b', facecolor='none')
    a1.add_patch(rect)

plt.imshow(img, cmap='gray')
a1.axis('off')

# Overlay mask over the drr image.
a2 = plt.subplot(1, 3, 3)
img_norm = np.array(img)
mask_img = Image.open(glob(os.path.join('./data/original_final/mask_all', check_file.replace('_img.png', '') + '*.png'))[0])
mask_img = np.array(mask_img)
mask_norm = mask_img[:, :, 0]

mask_overlay = np.ma.masked_where(mask_norm > 0, mask_norm)
img2mask = np.ma.masked_array(mask_norm, mask_overlay)

plt.imshow(img_norm, cmap='gray')
plt.imshow(img2mask, cmap='gray', interpolation='none', alpha=0.5)
a2.axis('off')

# Compare Detection Results (plot original, ground truth, prediction)

In [ ]:
test_file_path = './data/original_final/test'
result_file_path = './test_result/cp_faster_rcnn_resnet50_set1'

In [ ]:
want_to_check = list(map(os.path.basename, glob(os.path.join(result_file_path, '*.png'))))

In [ ]:
check_file = want_to_check[0]

plt.figure(figsize=(20,20))

# Plot DRR Image without Nodules
a0 = plt.subplot(1, 3, 1)
plt.imshow(Image.open(os.path.join(original_file_path, check_file)), cmap='gray')
a0.axis(False)


# Plot DRR Image with Nodules
a1 = plt.subplot(1, 3, 2)
img = Image.open(os.path.join(original_file_path, check_file))

with open('./test_revised_opencxr.json') as json_read:
    json_data = json.load(json_read)

img_idx = -1
for idx in range(len(json_data['images'])):
    if json_data['images'][idx]['file_name'] == check_file:
        img_idx = json_data['images'][idx]['id']
        break

anno_idx = []
for idx in range(len(json_data['annotations'])):
    if json_data['annotations'][idx]['image_id'] == img_idx:
        anno_idx.append(idx)

for idx in anno_idx:
    row = json_data['annotations'][idx]
    bbox_x, bbox_y, bbox_w, bbox_h = row['bbox']
    rect = patches.Rectangle((bbox_x, bbox_y), bbox_w, bbox_h, linewidth=2, edgecolor='b', facecolor='none')
    a1.add_patch(rect)

plt.imshow(img, cmap='gray')
a1.axis('off')


# Plot predicted DRR Image with Nodules
a2 = plt.subplot(1, 3, 3)
plt.imshow(Image.open(os.path.join(result_file_path, check_file)), cmap='gray')
a2.axis('off')
plt.show()